In [1]:
import tensorflow as tf
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from ProcesamientoCNN import extractorDataset, processImage
from ProcesamientoCNN import build_complete_siamese_network,contrastive_loss

In [3]:
inputData=(150,220) #150,220

In [4]:
model = build_complete_siamese_network((inputData[0], inputData[1], 1))  # Tamaño de imagen ejemplo
model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.001))

In [5]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 220, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 150, 220, 1  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 4096)         35326276    ['input_2[0][0]',                
                                                                  'input_3[0][0]']          

In [6]:
# Usar el generador para el entrenamiento
train_generator = extractorDataset("./train/", inputData, 64)
# Usar el generador para la validacion
val_generator = extractorDataset("./val/", inputData, 64)

In [7]:
""" model.load_weights("weights/")#25 #61
model.evaluate(val_generator,steps=100) """

' model.load_weights("weights/")#25 #61\nmodel.evaluate(val_generator,steps=100) '

In [7]:
print("GPUs available:", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs available: 1


In [8]:
# Entrenar el modelo
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./weights/",
                                                save_weights_only=True,
                                                mode='min',                           
                                                save_best_only=True,
                                                monitor='val_loss',
                                                verbose=1)
# Definir el callback EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear
    patience=10,         # Número de épocas sin mejora antes de detener
    verbose=1,           # Verbosidad
    mode='min',          # Modo 'min' para detener cuando la métrica deja de disminuir
    restore_best_weights=True  # Restaurar los mejores pesos al final del entrenamiento
)
history=model.fit(train_generator,validation_data=val_generator,validation_steps=100, epochs=75,steps_per_epoch=320,callbacks=[cp_callback,early_stopping])

Epoch 1/75
320/320 [==============================] - ETA: 0s - loss: 995.6195
Epoch 1: val_loss improved from inf to 1343.94873, saving model to ./weights\
320/320 [==============================] - 139s 392ms/step - loss: 995.6195 - val_loss: 1343.9487
Epoch 2/75
320/320 [==============================] - ETA: 0s - loss: 517.6062
Epoch 2: val_loss improved from 1343.94873 to 257.65292, saving model to ./weights\
320/320 [==============================] - 118s 371ms/step - loss: 517.6062 - val_loss: 257.6529
Epoch 3/75
320/320 [==============================] - ETA: 0s - loss: 113.1629
Epoch 3: val_loss improved from 257.65292 to 2.65142, saving model to ./weights\
320/320 [==============================] - 132s 414ms/step - loss: 113.1629 - val_loss: 2.6514
Epoch 4/75
320/320 [==============================] - ETA: 0s - loss: 140.9941
Epoch 4: val_loss did not improve from 2.65142
320/320 [==============================] - 117s 366ms/step - loss: 140.9941 - val_loss: 31.5192
Epoch 5/

KeyboardInterrupt: 

In [10]:
imagenA="val/049/01_049.png"
imagenB="val/049_forg/01_0114049.png"
a=processImage(imagenA,inputData) 
b=processImage(imagenB,inputData) 
prediction=model.predict([np.array([a]),np.array([b])])[0][0]
print(f'a shape={a.shape}, dtype={a.dtype}')
print(f'b shape={b.shape}, dtype={b.dtype}')
print(f'{prediction}')
fig, axes = plt.subplots(1, 2)
# Mostrar la primera imagen en el subplot 1
axes[0].imshow(a,cmap='gray')
axes[0].axis('off')  # Opcional: Oculta los ejes
# Mostrar la segunda imagen en el subplot 2
axes[1].imshow(b,cmap='gray')
axes[1].axis('off')
plt.show()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
